In [ ]:
#default_exp online

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Online

> Online window operations

In [ ]:
#export
from math import ceil, sqrt
from typing import Callable, List, Optional, Union

import numpy as np

from window_ops.expanding import *
from window_ops.ewm import *
from window_ops.rolling import *
from window_ops.utils import _rolling_std

In [ ]:
np.random.seed(0)
y = np.random.rand(100)

In [ ]:
def test_online(OnlineOp, regular_op, y, n_updates=10, **op_kwargs):
    expected = regular_op(y, **op_kwargs)
    online = OnlineOp(**op_kwargs)
    calculated = online.fit_transform(y[:-n_updates]).tolist()
    for i in range(y.size - n_updates, y.size):
        calculated.append(online.update(y[i]))
    assert np.allclose(calculated, expected, equal_nan=True)

## Rolling

### Regular

In [ ]:
#exporti
class BaseOnlineRolling:
    
    def __init__(self, rolling_op: Callable, window_size: int, min_samples: Optional[int] = None):
        self.rolling_op = rolling_op
        self.window_size = window_size
        self.min_samples = min_samples
        
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        self.x = x[-self.window_size:].tolist()
        return self.rolling_op(x, self.window_size, self.min_samples)
    
    def _update_x(self, x: np.ndarray) -> None:
        self.x = self.x[1:] + [x]

In [ ]:
#export
class RollingMean(BaseOnlineRolling):
    
    def __init__(self, window_size: int, min_samples: Optional[int] = None):
        super().__init__(rolling_mean, window_size, min_samples)
    
    def update(self, x: float) -> float:
        self._update_x(x)
        return sum(self.x) / self.window_size

In [ ]:
test_online(RollingMean, rolling_mean, y, window_size=3, min_samples=1)

In [ ]:
#export
class RollingMax(BaseOnlineRolling):
    
    def __init__(self, window_size: int, min_samples: Optional[int] = None):
        super().__init__(rolling_max, window_size, min_samples)
    
    def update(self, x) -> float:
        self._update_x(x)
        return max(self.x)

In [ ]:
test_online(RollingMax, rolling_max, y, window_size=4, min_samples=1)

In [ ]:
#export
class RollingMin(BaseOnlineRolling):
    
    def __init__(self, window_size, min_samples):
        super().__init__(rolling_min, window_size, min_samples)
    
    def update(self, x) -> float:
        self.x = self.x[1:] + [x]
        return min(self.x)

In [ ]:
test_online(RollingMin, rolling_min, y, window_size=4, min_samples=1)

In [ ]:
#export
class RollingStd(BaseOnlineRolling):
    
    def __init__(self, window_size, min_samples):
        super().__init__(rolling_std, window_size, min_samples)
    
    def update(self, x) -> float:
        self._update_x(x)
        return np.std(self.x, ddof=1)

In [ ]:
test_online(RollingStd, rolling_std, y, window_size=4, min_samples=2)

### Seasonal

In [ ]:
#exporti
class BaseOnlineSeasonalRolling:
    
    def __init__(self,
                 seasonal_rolling_op: Callable,
                 season_length: int,
                 window_size: int,
                 min_samples: Optional[int] = None):
        self.seasonal_rolling_op = seasonal_rolling_op
        self.season_length = season_length
        self.window_size = window_size
        self.min_samples = min_samples
        
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        n_samples = x.size
        min_samples = self.season_length * self.window_size
        if n_samples < min_samples:
            return np.full(n_samples, np.nan)
        self.x = x[-min_samples:].tolist()
        return self.seasonal_rolling_op(x, self.season_length, self.window_size, self.min_samples)
    
    def _update_and_get_seasonal_terms(self, x: float) -> List[float]:
        self.x = self.x[1:] + [x]
        return self.x[self.season_length-1::self.season_length]

In [ ]:
#export
class SeasonalRollingMean(BaseOnlineSeasonalRolling):
    
    def __init__(self,
                 season_length: int,
                 window_size: int,
                 min_samples: Optional[int] = None):
        super().__init__(seasonal_rolling_mean, season_length, window_size, min_samples)
    
    def update(self, x: float) -> float:
        seasonal_terms = self._update_and_get_seasonal_terms(x)
        return sum(seasonal_terms) / self.window_size

In [ ]:
test_online(SeasonalRollingMean, seasonal_rolling_mean, y, window_size=4, min_samples=1, season_length=7)

In [ ]:
#export
class SeasonalRollingStd(BaseOnlineSeasonalRolling):
    
    def __init__(self,
                 season_length: int,
                 window_size: int,
                 min_samples: Optional[int] = None):
        super().__init__(seasonal_rolling_std, season_length, window_size, min_samples)
    
    def update(self, x: float) -> float:
        seasonal_terms = self._update_and_get_seasonal_terms(x)
        return np.std(seasonal_terms, ddof=1)

In [ ]:
test_online(SeasonalRollingStd, seasonal_rolling_std, y, window_size=4, min_samples=2, season_length=7)

In [ ]:
#export
class SeasonalRollingMin(BaseOnlineSeasonalRolling):
    
    def __init__(self,
                 season_length: int,
                 window_size: int,
                 min_samples: Optional[int] = None):
        super().__init__(seasonal_rolling_min, season_length, window_size, min_samples)
    
    def update(self, x: float) -> float:
        seasonal_terms = self._update_and_get_seasonal_terms(x)
        return min(seasonal_terms)

In [ ]:
test_online(SeasonalRollingMin, seasonal_rolling_min, y, window_size=4, min_samples=1, season_length=7)

In [ ]:
#export
class SeasonalRollingMax(BaseOnlineSeasonalRolling):
    
    def __init__(self,
                 season_length: int,
                 window_size: int,
                 min_samples: Optional[int] = None):
        super().__init__(seasonal_rolling_max, season_length, window_size, min_samples)
    
    def update(self, x: float) -> float:
        seasonal_terms = self._update_and_get_seasonal_terms(x)
        return max(seasonal_terms)

In [ ]:
test_online(SeasonalRollingMax, seasonal_rolling_max, y, window_size=4, min_samples=1, season_length=7)

## Expanding

### Regular

In [ ]:
#export
class ExpandingMean:
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        exp_mean = expanding_mean(x)
        self.n = x.size
        self.cumsum = exp_mean[-1] * self.n
        return exp_mean
        
    def update(self, x: float) -> float:
        self.cumsum += x
        self.n += 1
        return self.cumsum / self.n

In [ ]:
np.random.seed(0)
y = np.random.rand(100)

In [ ]:
test_online(ExpandingMean, expanding_mean, y)

In [ ]:
#export
class ExpandingMax:
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        exp_max = expanding_max(x)
        self.max = exp_max[-1]
        return exp_max
        
    def update(self, x: float) -> float:
        if x > self.max:
            self.max = x
        return self.max

In [ ]:
test_online(ExpandingMax, expanding_max, y)

In [ ]:
#export
class ExpandingMin:
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        exp_min = expanding_min(x)
        self.min = exp_min[-1]
        return exp_min
        
    def update(self, x: float) -> float:
        if x < self.min:
            self.min = x
        return self.min

In [ ]:
test_online(ExpandingMin, expanding_min, y)

In [ ]:
#export
class ExpandingStd:
    
    def fit_transform(self, x):
        self.n = x.size
        exp_std, self.curr_avg, self.x_m2n = _rolling_std(x,
                                                          window_size=self.n,
                                                          min_samples=2)
        return exp_std
    
    def update(self, x):
        prev_avg = self.curr_avg
        self.n += 1
        self.curr_avg = prev_avg + (x - prev_avg) / self.n
        self.x_m2n += (x - prev_avg) * (x - self.curr_avg)
        return sqrt(self.x_m2n / (self. n - 1))

In [ ]:
test_online(ExpandingStd, expanding_std, y)

### Seasonal

In [ ]:
#exporti
class BaseSeasonalExpanding:

    def __init__(self,
                 ExpandingOp: Union[ExpandingMean, ExpandingMax, ExpandingMin, ExpandingStd],
                 season_length: int):
        self.ExpandingOp = ExpandingOp
        self.season_length = season_length
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        self.expanding_ops = []
        self.n_samples = x.size
        result = np.empty(self.n_samples)
        for season in range(self.season_length):
            exp_op = self.ExpandingOp()
            result[season::self.season_length] = exp_op.fit_transform(x[season::self.season_length])
            self.expanding_ops.append(exp_op)
        return result
    
    def update(self, x: float) -> float:
        season = self.n_samples % self.season_length        
        self.n_samples += 1
        return self.expanding_ops[season].update(x)    

In [ ]:
#export
class SeasonalExpandingMean(BaseSeasonalExpanding):
    
    def __init__(self, season_length):
        super().__init__(ExpandingMean, 7)

In [ ]:
test_online(SeasonalExpandingMean, seasonal_expanding_mean, y, season_length=7)

In [ ]:
#export
class SeasonalExpandingStd(BaseSeasonalExpanding):
    
    def __init__(self, season_length):
        super().__init__(ExpandingStd, 7)

In [ ]:
test_online(SeasonalExpandingStd, seasonal_expanding_std, y, season_length=7)

In [ ]:
#export
class SeasonalExpandingMin(BaseSeasonalExpanding):
    
    def __init__(self, season_length):
        super().__init__(ExpandingMin, 7)

In [ ]:
test_online(SeasonalExpandingMin, seasonal_expanding_min, y, season_length=7)

In [ ]:
#export
class SeasonalExpandingMax(BaseSeasonalExpanding):
    
    def __init__(self, season_length):
        super().__init__(ExpandingMax, 7)

In [ ]:
test_online(SeasonalExpandingMax, seasonal_expanding_max, y, season_length=7)

## EWM

In [ ]:
#export
class EWMMean:
    
    def __init__(self, alpha):
        self.alpha = alpha
        
    def fit_transform(self, x):
        mn = ewm_mean(x, self.alpha)
        self.smoothed = mn[-1]
        return mn
    
    def update(self, x):
        self.smoothed = self.alpha * x + (1 - self.alpha) * self.smoothed
        return self.smoothed

In [ ]:
test_online(EWMMean, ewm_mean, y, alpha=0.3)